# Flask SQLAlchemy ORM - Praca z danymi (komendy DML i DQL)

**Definicje modelu**

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>

In [3]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass

db = SQLAlchemy(model_class=Base)

# create the app
app = Flask(__name__)

# configure the SQLite database, relative to the app instance folder
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.sqlite3"

# initialize the app with the extension
db.init_app(app)

from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column

class Task(db.Model):
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String)

app.app_context().push()

## U - UPDATE (CRUD) - DML (Data Manipulation Language)

### Klauzula UPDATE

In [4]:
# Metoda I - metoda update klasy Query
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE ZMIENIĆ WSZYSTKICH wpisów 
# w tabeli.

tasks = Task.query.filter_by(name="Szukanie").update({'name': 'Znajdywanie'})
print(tasks)
db.session.commit()

2


Widzimy, że metoda update nie zwraca nam obiektu klasy Query tylko liczbę zmodyfikowanych wpisów.

In [5]:
# Metoda II - bezpośrednia modyfikacja wartości atrybutu instancji modelu

task = Task.query.filter_by(name="Znajdywanie").first()
task.name = "Szukanie"
db.session.commit()

## D - DELETE (CRUD) - DML (DATA Manipulation Language)

### Klauzula DELETE

In [6]:
# Metoda I - metoda delete klasy Query
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE USUNĄĆ WSZYSTKICH wpisów 
# w tabeli.

task = Task.query.filter_by(name="Pływanie").delete()
print(task)
db.session.commit()

1


Podobnie jak metoda update, metoda delete nie zwraca nam obiektu klasy Query tylko informacje o liczbie usuniętych wpisów

In [7]:
# Można też w inny sposób

task = Task.query.filter_by(name="Szukanie").first()
db.session.delete(task)
db.session.commit()

W ten sposób wywołujemy polecenia DML i DDL przy użyciu SQLAlchemy ORM. 

To samo możemy zrobić za pomocą SQLAlchemy Core.

## Core

Kilka przykładów

In [8]:
# C z CRUD
from sqlalchemy import insert

stmt = (
    insert(Task).
    values(name='Suszenie')
)
db.session.execute(stmt)
db.session.commit()

In [9]:
# R Z CRUD
from sqlalchemy import select

stmt = select(Task)
print(stmt)

SELECT task.id, task.name 
FROM task


In [10]:
result = db.session.execute(stmt)
print(result)

for item in result:
    print(item)

(<Task 1>,)
(<Task 2>,)
(<Task 3>,)
(<Task 4>,)
(<Task 6>,)
(<Task 7>,)
(<Task 9>,)
(<Task 10>,)
(<Task 11>,)


In [11]:
# fetchall

result = db.session.execute(stmt)
print(result.fetchall())

[(<Task 1>,), (<Task 2>,), (<Task 3>,), (<Task 4>,), (<Task 6>,), (<Task 7>,), (<Task 9>,), (<Task 10>,), (<Task 11>,)]


In [12]:
# fetchmany

result = db.session.execute(stmt)
print(result.fetchmany(size=5))

[(<Task 1>,), (<Task 2>,), (<Task 3>,), (<Task 4>,), (<Task 6>,)]


In [13]:
# fetchaone

result = db.session.execute(stmt)
print(result.fetchone())

(<Task 1>,)


### Filtry

In [14]:
stmt = (
    select(Task.name).
    where(Task.id==4)
)
task = db.session.execute(stmt)
print(task.fetchall())

[('Malowanie',)]


In [15]:
stmt = (
    select(Task).
    filter_by(name="Szukanie")
)
task = db.session.execute(stmt).fetchall()
print(task)

[]


### Update

In [16]:
# U z CRUD
from sqlalchemy import update
stmt = (
    update(Task).
    where(Task.id == 2).
    values(name='Odśnieżanie')
)
db.session.execute(stmt)
db.session.commit()

### Delete

In [17]:
# D Z CRUD
from sqlalchemy import delete

stmt = (
    delete(Task).
    where(Task.id == 2)
)
db.session.execute(stmt)
db.session.commit()

### Wykonywanie surowego sql

In [18]:
stmt = db.text('select * from task;')
cursor = db.session.execute(stmt)

# result as a list of tuples
result = cursor.fetchall()
print(result)

[(1, 'Sprzątanie'), (3, 'Zamiatanie'), (4, 'Malowanie'), (6, 'Znajdywanie'), (7, 'Programowanie'), (9, 'Pranie'), (10, 'Dodawanie'), (11, 'Suszenie')]


In [19]:
stmt = db.text('select * from task;')
cursor = db.session.execute(stmt)

# result as a list of dicts
result = cursor.mappings().fetchall()
print(result)

[{'id': 1, 'name': 'Sprzątanie'}, {'id': 3, 'name': 'Zamiatanie'}, {'id': 4, 'name': 'Malowanie'}, {'id': 6, 'name': 'Znajdywanie'}, {'id': 7, 'name': 'Programowanie'}, {'id': 9, 'name': 'Pranie'}, {'id': 10, 'name': 'Dodawanie'}, {'id': 11, 'name': 'Suszenie'}]
